In [ ]:
from datasets import load_dataset
import pandas as pd
import logging
import time
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
import os
from langchain_cohere import ChatCoh
ere

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from IPython.display import display, HTML
import json

In [3]:
file_path = '../../datasets/question answering/belebele/pes_arab_dataset.csv'  # Path for the CSV file
data = pd.read_csv(file_path)

# Sample 50 entries from the dataset
sampled_data = data.sample(n=50, random_state=42)

sampled_data

,link,question_number,flores_passage,question,mc_answer1,mc_answer2,mc_answer3,mc_answer4,correct_answer_num,dialect,ds
70,https://en.wikibooks.org/wiki/High_School_Eart...,1,بشر هزاران هزار سال از عدسی‌ها برای بزرگ‌نمایی...,کدام یک از موارد زیر را انسان‌ها در قرن شانزده...,استفاده از عدسی‌ها,بزرگنمایی اشیاء,ساخت عدسی‌ها,ساخت تلسکوپ,4,pes_Arab,2023-06-07
827,https://en.wikivoyage.org/wiki/Paraguay,1,گوارانی‌ها مهم‌ترین گروه بومی ساکن پاراگوئه شر...,دوران استعمارگری که سه قرن طول کشید توسط کدام ...,پاراگوئه,Guaycurú,گورانی‌ها,فاتحان اسپانیایی,4,pes_Arab,2023-06-07
231,https://en.wikibooks.org/wiki/Wikijunior:Solar...,2,خورشید پوسته‌ای شبیه کره زمین ندارد که بتوانید...,با توجه به متن، کدام قسمت از خورشید از زمین قا...,پوسته,پلاسما,فوتوسفر,مرکز,3,pes_Arab,2023-06-07
588,https://en.wikinews.org/wiki/Seven_killed_in_C...,2,آن مرد ظاهراً یک خودروی سه‌چرخ مسلح به مواد من...,مقامات چه اطلاعاتی درخصوص حمله تأیید کرده بودند؟,هویت هم‌دست‌ها,قومیت مظنون,انگیزه حمله,نام و نام خانوادگی مظنون,2,pes_Arab,2023-06-07
39,https://en.wikibooks.org/wiki/Environmental_th...,1,ما می توانیم زندگی دوستانه تری را با محیط زیست...,با توجه به متن، کدام مورد راه‌حلی قابل اجرا بر...,تحریم انتخاب‌های سبک زندگی مصرف‌گرایانه,کنشگری,یافتن ریشه مسئله,تبدیل شدن به متحد جنبش محیط زیست,3,pes_Arab,2023-06-07
731,https://en.wikivoyage.org/wiki/Downhill_snowsp...,2,ورزش‌های زمستانی روی تپه، از جمله اسکی و اسنوب...,مطابق با متن، حدود چه زمانی اسکی‌بازی اولین با...,قرن هجدهم,5000 سال قبل از میلاد,1861,قرن هفدهم,4,pes_Arab,2023-06-07
299,https://en.wikinews.org/wiki/Australian_Big_Br...,2,ACMA همچنین نتیجه گرفت که علیرغم اینکه این وید...,برای اینکه رسانه تحت مقوله محتوای اینترنتی لحا...,باید از طریق اینترنت پخش شود,باید به صورت فیزیکی در سروری قرار گیرد,نمی‌تواند روی وبسایت ذخیره شود,باید توسط ACMA تأیید شود,2,pes_Arab,2023-06-07
110,https://en.wikibooks.org/wiki/Mysticism/What_i...,2,عرفان عبارت است از جستجوی ارتباط، شناخت یا درک...,با توجه به متن، کدام یک از موارد زیر مورد توجه...,جستجوی شهود به واقعیت الهی,طلب بینش از طریق تجربیات مشترک و همگانی,تأکید بر حالت سعادتمند خودآگاه,طلب درک آگاهانه الوهیت,2,pes_Arab,2023-06-07
72,https://en.wikibooks.org/wiki/History_of_Islam...,1,محمد عمیقا به امور فراتر از این زندگی دنیوی عل...,بر اساس متن، غار محمد بیانگر چه چیزی درباره اع...,او برای آرامش و انزوا ارزش قائل بود,صدای بیرون به تأمل او کمک می‌کرد,به زمان خیلی کمی برای تنهایی نیاز داشت,نگران این بود که دیده یا شنیده شود,1,pes_Arab,2023-06-07
86,https://en.wikibooks.org/wiki/Introduction_to_...,1,کودکان از زمان کودکی نسبت به نژاد و کلیشه های ...,با توجه به نظریه ارائه شده در متن، برای کودکان...,کودک سعی می‌کند از درگیر شدن در رفتارهای تهدید...,احتمال بیشتری وجود دارد که کودک درگیر رفتارهای...,کودک در موقعیت‌های اجتماعی کناره‌گیری می‌کند,کودک از تعامل با سایر اقلیت‌های نژادی امتناع خ...,2,pes_Arab,2023-06-07


In [5]:
os.environ['COHERE_API_KEY'] = 'k4nv2XHVUMI9IDp3R4TV5jyzax5DufqG1nZDh5up'

In [ ]:
import cohere

# Initialize Cohere client with your API key
co = cohere.ClientV2(log_warning_experimental_features=False)

In [ ]:
# Function to predict the answer and save the full generation to a file
def predict_answer_and_save(passage, question, options, cohere_object):
    logging.info(f"Starting prediction for passage: {passage[:50]}... and question: {question}")
    try:
        # Prepare the messages for the model
        res = cohere_object.chat(
            model="command-r-plus",  # Replace with the correct model
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You will receive a `QUESTION` with multiple `ANSWER CHOICES` and a supporting `CONTEXT`. "
                        "Your task is to determine the correct answer based on the information in the context and to explain "
                        "your reasoning step-by-step, referencing key details from the context.\n\n"
                        "Follow these instructions for your response:\n\n"
                        "Select only one answer choice (1, 2, 3, or 4) based on your understanding of the context.\n"
                        "Provide a detailed, step-by-step explanation of how you arrived at the answer, focusing on the logical reasoning process. "
                        "Include references to relevant parts of the context to support each reasoning step. `The explanation must be in Persian.`\n"
                        "Identify and quote the most relevant parts of the context that were essential in answering the question. "
                        "Only include the specific information that directly supports your chosen answer. `Ensure this is in Persian.`\n"
                        "List the most important keywords from the context that are essential for understanding and answering the question. "
                        "---\n\n"
                        "`Guidelines for Explanation`:\n"
                        "- Make sure your explanation is complete, covering all reasoning steps used to reach the answer.\n"
                        "- Ensure your reasoning is relevant and directly tied to the context, avoiding any unrelated details.\n"
                        "- Keep the explanation clear and logical, with each step following naturally from the previous one.\n"
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"CONTEXT:\n{passage}\n\n"
                        f"QUESTION:\n{question}\n\n"
                        f"ANSWER CHOICES:\n1-{options[0]}\n2-{options[1]}\n3-{options[2]}\n4-{options[3]}"
                    )
                }
            ],
            response_format={
                "type": "json_object",
                "schema": {
                    "type": "object",
                    "properties": {
                        "answer": {"type": "integer"},
                        "explanation": {"type": "string"},
                        "highlight": {"type": "string"},
                        "keywords": {"type": "array", "items": {"type": "string"}}
                    },
                    "required": ["answer", "explanation", "highlight", "keywords"]
                },
            },
        )
        
        # Assuming the response is formatted as expected
        response = res.message.content[0].text.strip()
        
        # Output the model's response for preview
        print("Response from model:", response)
        
        # Parse the response into a JSON-like object (if it's in the correct format)
        response_json = json.loads(response)
        
        # Example: Print the response as a dictionary
        print("Parsed response:", response_json)
        return response_json
        
    except Exception as e:
        logging.error(f"Error during prediction: {e}")
        return None


2025-02-20 17:19:34,758 - Starting prediction for passage: The sun provides light and energy to the plants, w... and question: What is the primary role of the sun for plants?
2025-02-20 17:19:38,522 - HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"


Response from model: {
"answer": 1,
"explanation": "در متن ارائه شده، به صراحت بیان شده است که خورشید «نور و انرژی» را برای گیاهان فراهم می‌کند. نور خورشید یکی از عوامل حیاتی در فرآیند فتوسنتز است که گیاهان از آن برای تولید غذا و رشد استفاده می‌کنند. بنابراین، نقش اصلی خورشید در مورد گیاهان، «تامین نور» است. این نور برای انجام فرآیندهای حیاتی در گیاهان ضروری است. ",
"highlight": "The sun provides light and energy to the plants, which is essential for photosynthesis.",
"keywords": ["نور خورشید", "فتوسنتز", "انرژی"]
}
Parsed response: {'answer': 1, 'explanation': 'در متن ارائه شده، به صراحت بیان شده است که خورشید «نور و انرژی» را برای گیاهان فراهم می\u200cکند. نور خورشید یکی از عوامل حیاتی در فرآیند فتوسنتز است که گیاهان از آن برای تولید غذا و رشد استفاده می\u200cکنند. بنابراین، نقش اصلی خورشید در مورد گیاهان، «تامین نور» است. این نور برای انجام فرآیندهای حیاتی در گیاهان ضروری است. ', 'highlight': 'The sun provides light and energy to the plants, which is essential for photosynthesis.', 

In [19]:
# Sample data to test the function
passage = "The sun provides light and energy to the plants, which is essential for photosynthesis."
question = "What is the primary role of the sun for plants?"
options = ["A: Provides light", "B: Provides water", "C: Provides nutrients", "D: Provides oxygen"]

# Run the prediction function
result = predict_answer_and_save(passage, question, options, co)

# Check the result
if result:
    print("Predicted Answer:", result["answer"])
    print("Explanation:", result["explanation"])
    print("Highlight:", result["highlight"])
    print("Keywords:", result["keywords"])
else:
    print("Prediction failed.")

2025-02-20 17:30:44,252 - Starting prediction for passage: The sun provides light and energy to the plants, w... and question: What is the primary role of the sun for plants?
2025-02-20 17:30:48,922 - HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"


Response from model: {
"answer": 1,
"explanation": "در متن ارائه شده، به صراحت بیان شده است که خورشید «نور» و «انرژی» به گیاهان می‌رساند. نور خورشید یکی از عوامل ضروری برای انجام فرآیند فتوسنتز در گیاهان است. اگرچه آب و مواد مغذی نیز برای رشد و حیات گیاهان ضروری هستند، اما متن هیچ اشاره‌ای به نقش خورشید در تأمین آب یا مواد مغذی نکرده است. همچنین، گیاهان از طریق فرآیند فتوسنتز، اکسیژن تولید می‌کنند، اما خورشید به طور مستقیم در این فرآیند تولید اکسیژن دخیل نیست. بنابراین، نور خورشید نقش اصلی خورشید برای گیاهان محسوب می‌شود. ",
"highlight": "The sun provides light and energy to the plants, which is essential for photosynthesis.",
"keywords": ["نور خورشید", "فتوسنتز", "انرژی", "گیاهان"]
}
Parsed response: {'answer': 1, 'explanation': 'در متن ارائه شده، به صراحت بیان شده است که خورشید «نور» و «انرژی» به گیاهان می\u200cرساند. نور خورشید یکی از عوامل ضروری برای انجام فرآیند فتوسنتز در گیاهان است. اگرچه آب و مواد مغذی نیز برای رشد و حیات گیاهان ضروری هستند، اما متن هیچ اشاره\u200cای به نقش خور

In [17]:
def make_request_with_retry(func, *args, retries=3, delay=10):
    for attempt in range(retries):
        try:
            return func(*args)
        except ConnectionError as e:
            logging.warning(f"Attempt {attempt + 1} failed: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
    logging.error("Max retries exceeded")
    raise ConnectionError("Max retries exceeded")

In [ ]:
def apply_prediction_and_save(df, mode="predict_and_explain", result_name = 'sample'):
    results = []

    for index, row in df.iterrows():
        passage = row['flores_passage']
        question = row['question']
        options = [row['mc_answer1'], row['mc_answer2'], row['mc_answer3'], row['mc_answer4']]
        original_answer = row['correct_answer_num']

        logging.info(f"Processing row {index+1}/{len(df)}: {passage[:50]}...")

        # Select the appropriate method
        if mode == "just_predict":
            result = make_request_with_retry(predict_answer_and_save, passage, question, options, co)
            time.sleep(10)
        elif mode == "predict_and_explain":
            result = make_request_with_retry(predict_answer_and_save, passage, question, options, co)
            time.sleep(10)
        else:
            logging.error(f"Invalid mode: {mode}")
            predicted_answer = "error"

        results.append({
            "Passage": passage,
            "Question": question,
            "Original Answer": original_answer,
            "Predicted Answer": result["answer"],
            "Option 1": options[0],
            "Option 2": options[1],
            "Option 3": options[2],
            "Option 4": options[3],
            "Explanation": result["explanation"],
            "Highlight": result["highlight"],
            "Keywords":result["keywords"]
        })

        logging.info(f"Row {index+1} processed successfully.")

    # Save final results as DataFrame
    result_df = pd.DataFrame(results)
    output_file_name = result_name + ".csv"
    result_df.to_csv(output_file_name, index=False)
    logging.info("Results saved to {o}".format(o = output_file_name))

    return result_df

In [25]:
apply_prediction_and_save(sampled_data, mode='predict_and_explain', result_name='sample_result')

2025-02-20 17:34:49,492 - Processing row 71/50: بشر هزاران هزار سال از عدسی‌ها برای بزرگ‌نمایی است...
2025-02-20 17:34:49,494 - Starting prediction for passage: بشر هزاران هزار سال از عدسی‌ها برای بزرگ‌نمایی است... and question: کدام یک از موارد زیر را انسان‌ها در قرن شانزدهم شروع کردند؟
2025-02-20 17:34:53,568 - HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"


Response from model: {
"answer": 4,
"explanation": "در متن به صراحت اشاره شده است که ساخت تلسکوپ های واقعی در قرن شانزدهم در اروپا آغاز شد. اگرچه استفاده از عدسی ها و بزرگنمایی اشیاء سابقه ای طولانی تر دارد، اما ساخت تلسکوپ به عنوان یک ابزار اپتیکی پیشرفته، در این دوره زمانی شروع شده است. این تلسکوپ ها با ترکیب دو عدسی، توانایی نمایش اجسام دور را به صورت بزرگتر و نزدیکتر فراهم می کردند. بنابراین، گزینه 4، «ساخت تلسکوپ»، پاسخ صحیح است. ",
"highlight": "با این حال، اولین تلسکوپ های واقعی در اواخر قرن 16 در اروپا ساخته شدند.",
"keywords": ["عدسی", "بزرگنمایی", "تلسکوپ", "قرن شانزدهم", "ساخت"]
}
Parsed response: {'answer': 4, 'explanation': 'در متن به صراحت اشاره شده است که ساخت تلسکوپ های واقعی در قرن شانزدهم در اروپا آغاز شد. اگرچه استفاده از عدسی ها و بزرگنمایی اشیاء سابقه ای طولانی تر دارد، اما ساخت تلسکوپ به عنوان یک ابزار اپتیکی پیشرفته، در این دوره زمانی شروع شده است. این تلسکوپ ها با ترکیب دو عدسی، توانایی نمایش اجسام دور را به صورت بزرگتر و نزدیکتر فراهم می کردند. بنابراین، گزینه 

2025-02-20 17:35:03,578 - Row 71 processed successfully.
2025-02-20 17:35:03,578 - Processing row 828/50: گوارانی‌ها مهم‌ترین گروه بومی ساکن پاراگوئه شرقی ک...
2025-02-20 17:35:03,578 - Starting prediction for passage: گوارانی‌ها مهم‌ترین گروه بومی ساکن پاراگوئه شرقی ک... and question: دوران استعمارگری که سه قرن طول کشید توسط کدام گروه شروع شد؟
2025-02-20 17:35:08,629 - HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"


Response from model: {  "answer": 4,
  "explanation": "در متن به صراحت ذکر شده است که دوره استعماری سه قرنه توسط فاتحان اسپانیایی آغاز شد. این دوره از قرن 16 شروع شد و تا قرن 19 ادامه داشت. پاراگوئه در این دوره تحت تسلط امپراتوری اسپانیا بود و به عنوان یکی از مستعمرات آن شناخته می‌شد. گوارانی‌ها و سایر گروه‌های بومی در این دوره تحت تأثیر فرهنگ و زبان اسپانیایی قرار گرفتند و بسیاری از آداب و رسوم خود را از دست دادند. با این حال، برخی از سنت‌ها و زبان بومی تا حدودی حفظ شد. ",
  "highlight": "اسپانیایی‌ها دوره استعماری را شروع کردند که سه قرن طول کشید.",
  "keywords": ["فاتحان اسپانیایی", "دوره استعماری", "قرن 16", "مستعمره", "گوارانی‌ها"]
}
Parsed response: {'answer': 4, 'explanation': 'در متن به صراحت ذکر شده است که دوره استعماری سه قرنه توسط فاتحان اسپانیایی آغاز شد. این دوره از قرن 16 شروع شد و تا قرن 19 ادامه داشت. پاراگوئه در این دوره تحت تسلط امپراتوری اسپانیا بود و به عنوان یکی از مستعمرات آن شناخته می\u200cشد. گوارانی\u200cها و سایر گروه\u200cهای بومی در این دوره تحت تأثیر فرهنگ 

2025-02-20 17:35:18,643 - Row 828 processed successfully.
2025-02-20 17:35:18,644 - Processing row 232/50: خورشید پوسته‌ای شبیه کره زمین ندارد که بتوانید روی...
2025-02-20 17:35:18,644 - Starting prediction for passage: خورشید پوسته‌ای شبیه کره زمین ندارد که بتوانید روی... and question: با توجه به متن، کدام قسمت از خورشید از زمین قابل مشاهده است؟
2025-02-20 17:35:22,056 - HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"


...

Response from model: {  "answer": 3,
  "explanation": "در متن به صراحت ذکر شده است که کارولین ویلسون از OHA پول‌پیش خانواده‌ها را به سرقت برده است. این نشان می‌دهد که او قبل از فرار از شهر، در OHA مشغول به کار بوده است. همچنین، OHA به عنوان یک سازمان یا نهاد معرفی شده است، در حالی که باغ‌های لاک‌وود یک مکان جغرافیایی است و نمی‌تواند کارفرمای کارولین باشد. هیچ اشاره‌ای به ارتباط او با مرکز حقوق عام خلیج شرقی یا پلیس اوکلند در متن وجود ندارد. بنابراین، گزینه 3 تنها انتخاب منطقی است. ",
  "highlight": " کارولین ویلسون از OHA پول‌پیش خانواده‌ها را به سرقت برده است.",
  "keywords": ["کارولین ویلسون", "OHA", "سرقت", "پول‌پیش", "مرکز حقوق عام خلیج شرقی", "پلیس OHA", "باغ لاک وود"]
}
Parsed response: {'answer': 3, 'explanation': 'در متن به صراحت ذکر شده است که کارولین ویلسون از OHA پول\u200cپیش خانواده\u200cها را به سرقت برده است. این نشان می\u200cدهد که او قبل از فرار از شهر، در OHA مشغول به کار بوده است. همچنین، OHA به عنوان یک سازمان یا نهاد معرفی شده است، در حالی که باغ\u200cهای لاک\u200cو

2025-02-20 17:46:23,699 - Row 495 processed successfully.
2025-02-20 17:46:23,700 - Processing row 833/50: مسافران ممکن است با انواعی از حیوانات موذی مواجه ش...
2025-02-20 17:46:23,700 - Starting prediction for passage: مسافران ممکن است با انواعی از حیوانات موذی مواجه ش... and question: انجام کدام‌یک از موارد زیر احتمالاً از یک حیوان موذی بعید است؟
2025-02-20 17:46:28,517 - HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"


Response from model: {  "answer": 4,
  "explanation": "در متن ارائه شده، به صراحت ذکر شده است که حیوانات موذی ممکن است سبب انتقال عفونت شوند، اما اشاره‌ای به توانایی آنها در ایجاد جراحت‌های شدید نشده است. در واقع، حیوانات موذی معمولاً به دلیل اندازه و قدرت محدود خود، توانایی آسیب‌رسانی جدی به انسان را ندارند. بنابراین، گزینه 4، یعنی 'سبب جراحت‌های شدید شدن'، احتمالاً از یک حیوان موذی بعید است. ",
  "highlight": "حیوانات موذی غذا را خراب می‌کنند، سبب آزار و اذیت می‌شوند یا در حالت بدتر سبب واکنش‌های حساسیت‌زا می‌شوند، زهر خود را پخش می‌کنند یا باعث انتقال عفونت‌ها می‌شوند. خود بیماری‌های عفونی یا حیوانات خطرناکی که می‌توانند افراد را با قدرت خود مجروح کنند یا بکشند، معمولاً با عنوان بلایا توصیف نمی‌شوند.",
  "keywords": ["حیوانات موذی", "انتقال عفونت", "جراحت‌های شدید", "بیماری‌های عفونی", "حیوانات خطرناک"]
}
Parsed response: {'answer': 4, 'explanation': "در متن ارائه شده، به صراحت ذکر شده است که حیوانات موذی ممکن است سبب انتقال عفونت شوند، اما اشاره\u200cای به توانایی آنها در ایجاد جرا

2025-02-20 17:46:38,522 - Row 833 processed successfully.
2025-02-20 17:46:38,915 - Results saved to sample_result.csv


,Passage,Question,Original Answer,Predicted Answer,Option 1,Option 2,Option 3,Option 4,Explanation:,Highlight:,Keywords:
0,بشر هزاران هزار سال از عدسی‌ها برای بزرگ‌نمایی...,کدام یک از موارد زیر را انسان‌ها در قرن شانزده...,4,4,استفاده از عدسی‌ها,بزرگنمایی اشیاء,ساخت عدسی‌ها,ساخت تلسکوپ,در متن به صراحت اشاره شده است که ساخت تلسکوپ ه...,با این حال، اولین تلسکوپ های واقعی در اواخر قر...,"[عدسی, بزرگنمایی, تلسکوپ, قرن شانزدهم, ساخت]"
1,گوارانی‌ها مهم‌ترین گروه بومی ساکن پاراگوئه شر...,دوران استعمارگری که سه قرن طول کشید توسط کدام ...,4,4,پاراگوئه,Guaycurú,گورانی‌ها,فاتحان اسپانیایی,در متن به صراحت ذکر شده است که دوره استعماری س...,اسپانیایی‌ها دوره استعماری را شروع کردند که سه...,"[فاتحان اسپانیایی, دوره استعماری, قرن 16, مستع..."
2,خورشید پوسته‌ای شبیه کره زمین ندارد که بتوانید...,با توجه به متن، کدام قسمت از خورشید از زمین قا...,3,3,پوسته,پلاسما,فوتوسفر,مرکز,در متن به وضوح ذکر شده است که «وقتی به خورشید ...,وقتی به خورشید نگاه می‌کنیم قسمت بیرونی که می‌...,"[فوتوسفر, قسمت بیرونی, قابل مشاهده]"
3,آن مرد ظاهراً یک خودروی سه‌چرخ مسلح به مواد من...,مقامات چه اطلاعاتی درخصوص حمله تأیید کرده بودند؟,2,2,هویت هم‌دست‌ها,قومیت مظنون,انگیزه حمله,نام و نام خانوادگی مظنون,در متن ارائه شده، به دو مورد از اطلاعات تایید ...,مرد مظنون به انفجار بمب، پس از تحمل جراحات ناش...,"[مرد مظنون, بازداشت, گروه قومی اویغور]"
4,ما می توانیم زندگی دوستانه تری را با محیط زیست...,با توجه به متن، کدام مورد راه‌حلی قابل اجرا بر...,3,3,تحریم انتخاب‌های سبک زندگی مصرف‌گرایانه,کنشگری,یافتن ریشه مسئله,تبدیل شدن به متحد جنبش محیط زیست,در این متن، به دنبال راه‌حلی برای مسائل زیست‌م...,با این وجود، اگر تنها به دنبال یک راه‌حل موقت ...,"[راه‌حل موقت, ریشه مشکلات, فعالیت‌های محیط زیس..."
5,ورزش‌های زمستانی روی تپه، از جمله اسکی و اسنوب...,مطابق با متن، حدود چه زمانی اسکی‌بازی اولین با...,4,4,قرن هجدهم,5000 سال قبل از میلاد,1861,قرن هفدهم,در متن به صراحت اشاره شده است که ورزش اسکی روی...,ورزش اسکی روی تپه حداقل به قرن 17 ام برمی‌گردد,"[اسکی, ورزش, قرن 17 ام]"
6,ACMA همچنین نتیجه گرفت که علیرغم اینکه این وید...,برای اینکه رسانه تحت مقوله محتوای اینترنتی لحا...,2,2,باید از طریق اینترنت پخش شود,باید به صورت فیزیکی در سروری قرار گیرد,نمی‌تواند روی وبسایت ذخیره شود,باید توسط ACMA تأیید شود,در متن به صراحت بیان شده است که برای اینکه محت...,قانون خدمات پخش مقررات محتوای اینترنتی را مشخص...,"[قانون خدمات پخش, محتوای اینترنتی, سرور, ذخیره]"
7,عرفان عبارت است از جستجوی ارتباط، شناخت یا درک...,با توجه به متن، کدام یک از موارد زیر مورد توجه...,2,2,جستجوی شهود به واقعیت الهی,طلب بینش از طریق تجربیات مشترک و همگانی,تأکید بر حالت سعادتمند خودآگاه,طلب درک آگاهانه الوهیت,در متن به صراحت بیان شده است که عرفان بر تجربه...,در عرفان بر تجربه مستقیم **شخصی** تأکید دارد.,"[تجربه مستقیم, حالت هشیاری, تجربه شخصی, تجربیا..."
8,محمد عمیقا به امور فراتر از این زندگی دنیوی عل...,بر اساس متن، غار محمد بیانگر چه چیزی درباره اع...,1,1,او برای آرامش و انزوا ارزش قائل بود,صدای بیرون به تأمل او کمک می‌کرد,به زمان خیلی کمی برای تنهایی نیاز داشت,نگران این بود که دیده یا شنیده شود,متن به وضوح بیان می‌کند که غار حرا، مکانی منزو...,در یک انزوا کامل خواهید بود,"[غار حرا, انزوا, آرامش, تأمل]"
9,کودکان از زمان کودکی نسبت به نژاد و کلیشه های ...,با توجه به نظریه ارائه شده در متن، برای کودکان...,2,2,کودک سعی می‌کند از درگیر شدن در رفتارهای تهدید...,احتمال بیشتری وجود دارد که کودک درگیر رفتارهای...,کودک در موقعیت‌های اجتماعی کناره‌گیری می‌کند,کودک از تعامل با سایر اقلیت‌های نژادی امتناع خ...,در این متن، نظریه‌ای مطرح می‌شود که نشان می‌ده...,کودکان از زمان کودکی نسبت به نژاد و کلیشه های ...,"[کلیشه نژادی, تأثیر بر رفتار, اقلیت نژادی, تطب..."


# Groq

In [1]:
from groq import Groq